In [1]:
!pip install -U keras-tuner # for performing hyperparameter tuning
!pip install --upgrade tensorflow jax jaxlib # To ensure that we have the latest versions of TensorFlow and JAX installed.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 778.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 MB 4.1 MB/s eta 0:00:00
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.4.26+cuda12.cudnn89
    Uninstalling jaxlib-0.4.26+cuda12.cudnn89:
      Successfully uninstalled jaxlib-0.4.26+cuda12.cudnn89
  Attempting uninstall: jax
    Found existing installation: jax 0.4.26
    Uninstalling jax-0.4.26:
      Successfully uninstalled jax-0.4.26


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential
import tensorflow as tf
from tensorflow.keras.layers import Dense
import keras_tuner as kf
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

In [3]:
df = pd.read_csv('/content/Churn_Modelling.csv')

In [4]:
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [5]:
df.drop(columns = ['RowNumber'	,'CustomerId',	'Surname'], inplace=True)

In [6]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [7]:
df = pd.get_dummies(data = df,columns=['Geography','Gender'])
df

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,True,False,False,True,False
1,608,41,1,83807.86,1,0,1,112542.58,0,False,False,True,True,False
2,502,42,8,159660.80,3,1,0,113931.57,1,True,False,False,True,False
3,699,39,1,0.00,2,0,0,93826.63,0,True,False,False,True,False
4,850,43,2,125510.82,1,1,1,79084.10,0,False,False,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,0,True,False,False,False,True
9996,516,35,10,57369.61,1,1,1,101699.77,0,True,False,False,False,True
9997,709,36,7,0.00,1,0,1,42085.58,1,True,False,False,True,False
9998,772,42,3,75075.31,2,1,0,92888.52,1,False,True,False,False,True


In [8]:
x = df.drop(columns = ['Exited'])
y = df['Exited']

In [9]:
sc = StandardScaler()

x = sc.fit_transform(x)

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size = 0.20, random_state=1)

In [12]:
# Before hyperparameter tuning: initially use random parameters only for comparision
ann = Sequential()# random weights are initialized so every time you will get different accuracy and loss

ann.add(Dense(units = 10, activation = 'relu'))# hidden layer
ann.add(Dense(units = 1, activation = 'sigmoid'))# output layer

ann.compile(optimizer = 'adadelta', loss = 'binary_crossentropy', metrics =['accuracy'])

ann.fit(xtrain,ytrain,epochs = 100)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.7250 - loss: 0.5696
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7299 - loss: 0.5570
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.7261 - loss: 0.5617
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.7282 - loss: 0.5617
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.7284 - loss: 0.5622
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7402 - loss: 0.5559
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7289 - loss: 0.5611
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7283 - loss: 0.5666
Epoch 9/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.7251 - loss: 0.5680
Epoch 10/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.7250 - loss: 0.5715
Epoch 11/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.7326 - loss: 0.5626
Epoch 12/100
250/250 ━━━━━━━━━━━━━━━━━━

In [13]:
ypred = ann.predict(xtest)
ypred = ypred > 0.5
ypred

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [14]:
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.81      0.91      0.86      1585
           1       0.34      0.17      0.23       415

    accuracy                           0.76      2000
   macro avg       0.57      0.54      0.54      2000
weighted avg       0.71      0.76      0.73      2000



In [15]:
ann.evaluate(xtrain,ytrain)

250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7584 - loss: 0.5425


[0.5374243855476379, 0.7592499852180481]

# Hyperparameter tuning


In [17]:
list(range(8,128,8))

[8, 16, 24, 32, 40, 48, 56, 64, 72, 80, 88, 96, 104, 112, 120]

In [19]:
def hyper(hp):
    model = Sequential()

    # Add input layer
    model.add(Dense(hp.Int('units0', min_value=8, max_value=128, step=8),
                    activation=hp.Choice('activation0', values=['relu', 'tanh', 'sigmoid']),
                    input_dim=13))

    # Add additional hidden layers
    for i in range(1, hp.Int('num_layers', min_value=1, max_value=10)):
        model.add(Dense(hp.Int('units' + str(i), min_value=8, max_value=128, step=8),
                        activation=hp.Choice('activation' + str(i), values=['relu', 'tanh', 'sigmoid'])))

    # Add output layer
    model.add(Dense(units=1, activation='sigmoid'))

    # Compile the model
    model.compile(optimizer=hp.Choice('optimizer', values=['adam', 'rmsprop', 'sgd']),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

In [20]:
tuner = kf.RandomSearch(hyper,
                        objective = 'val_accuracy',
                        max_trials = 3)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [21]:
tuner.search(xtrain,ytrain,epochs = 3, validation_data = (xtest,ytest))
# search() is similar to fit()

Trial 3 Complete [00h 00m 11s]
val_accuracy: 0.8650000095367432

Best val_accuracy So Far: 0.8650000095367432
Total elapsed time: 00h 00m 29s


In [22]:
tuner.get_best_hyperparameters()[0].values # will give first model where accuracy is high
# Each of the 13 input features is connected to all 120 neurons in the input layer.
# This means that each of the 120 neurons will receive all 13 input values
# and apply a weighted sum and activation function to compute an output.

{'units0': 120,
 'activation0': 'relu',
 'num_layers': 8,
 'optimizer': 'adam',
 'units1': 72,
 'activation1': 'tanh',
 'units2': 104,
 'activation2': 'relu',
 'units3': 128,
 'activation3': 'relu',
 'units4': 72,
 'activation4': 'relu',
 'units5': 8,
 'activation5': 'tanh',
 'units6': 56,
 'activation6': 'sigmoid',
 'units7': 32,
 'activation7': 'relu',
 'units8': 80,
 'activation8': 'sigmoid'}

In [23]:
tuned_model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 38 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [24]:
tuned_model.fit(xtrain,ytrain,epochs=50, validation_data=(xtest,ytest))

Epoch 1/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8595 - loss: 0.3552 - val_accuracy: 0.8570 - val_loss: 0.3517
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.8641 - loss: 0.3276 - val_accuracy: 0.8625 - val_loss: 0.3382
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8561 - loss: 0.3428 - val_accuracy: 0.8670 - val_loss: 0.3329
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8613 - loss: 0.3352 - val_accuracy: 0.8715 - val_loss: 0.3376
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8601 - loss: 0.3371 - val_accuracy: 0.8605 - val_loss: 0.3455
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8669 - loss: 0.3241 - val_accuracy: 0.8670 - val_loss: 0.3382
Epoch 7/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8682 - loss: 0.3162 - val_accuracy: 0.8610 - val_loss: 0.3598
Epoch 8/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8691 - loss: 0.3210 - val_accuracy: 0.

In [25]:
tuned_model.evaluate(xtrain,ytrain) # Training accuracy

250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9300 - loss: 0.1717


[0.1626759022474289, 0.9352499842643738]

In [26]:
tuned_model.evaluate(xtest,ytest)# Testing accuracy

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8373 - loss: 0.4769


[0.5177645087242126, 0.8320000171661377]